# Optimizing Model Parameters

Training a model is an iterative process, in each iteration the model makes a guess about the output, calculates the error in its guess (loss), collects the derivatives of the error with espect to its parameters, and optimizes these parameters using gradient descent.

In [11]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

training_data = datasets.FashionMNIST(
    root="~/Developer/Datasets", 
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="~/Developer/Datasets",
    train=False,
    download=True,
    transform=ToTensor(),
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

print(next(iter(train_dataloader))[0].shape)
print(next(iter(train_dataloader))[0].shape)

Using mps device
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])


In [23]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()
model.to(device)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [24]:
# Hyperparameters
learning_rate = 1e-3
batch_size = 64
epochs = 5

loss_fn = nn.CrossEntropyLoss() # Initialize the loss function
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) # Initialize the optimizer

In [25]:
# Optimization Loop
def move_to_device(inputs, labels, device):
    return inputs.to(device), labels.to(device)

def train_loop(dataloader : DataLoader, model : nn.Module, loss_fn : torch.nn.modules.loss , optimizer):
    size = len(dataloader.dataset)
    model.train(mode=True)
    
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X, y = move_to_device(X, y, device)
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
      
def test_loop(dataloader : DataLoader, model : nn.Module, loss_fn : torch.nn.modules.loss):
    model.train(mode=False)
    model.eval()  # Important for batchnorm, dropout, etc.
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    # No need to compute gradients
    with torch.no_grad():
        for X, y in dataloader:
            X, y = move_to_device(X, y, device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
    
epochs = 10
for e in range(epochs):
    print(f"Epoch {e+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)

print("Done")  

Epoch 1
-------------------------------
loss: 2.274175 [    0/60000]
loss: 1.405884 [ 6400/60000]
loss: 0.939259 [12800/60000]
loss: 1.090776 [19200/60000]
loss: 0.869180 [25600/60000]
loss: 0.843452 [32000/60000]
loss: 0.772076 [38400/60000]
loss: 0.720589 [44800/60000]
loss: 0.744764 [51200/60000]
loss: 0.742828 [57600/60000]
Test Error: 
 Accuracy: 78.0%, Avg loss: 0.665286 

Epoch 2
-------------------------------
loss: 0.627443 [    0/60000]
loss: 0.699240 [ 6400/60000]
loss: 0.448176 [12800/60000]
loss: 0.742594 [19200/60000]
loss: 0.650697 [25600/60000]
loss: 0.614572 [32000/60000]
loss: 0.588853 [38400/60000]
loss: 0.612970 [44800/60000]
loss: 0.638653 [51200/60000]
loss: 0.602008 [57600/60000]
Test Error: 
 Accuracy: 81.0%, Avg loss: 0.551951 

Epoch 3
-------------------------------
loss: 0.506269 [    0/60000]
loss: 0.578354 [ 6400/60000]
loss: 0.364903 [12800/60000]
loss: 0.642414 [19200/60000]
loss: 0.569135 [25600/60000]
loss: 0.530781 [32000/60000]
loss: 0.504354 [38400/

In [26]:
torch.cuda.empty_cache()